In [ ]:
from smbus2 import SMBus, i2c_msg

with SMBus(1) as bus:   
    msg = i2c_msg.write(0x34, [ord('k'), ord('w'), 0xC1, 255, 5]) # bytes must be in range [0, 256)
    bus.i2c_rdwr(msg)

In [ ]:
from smbus2 import SMBus, i2c_msg

with SMBus(1) as bus:   
    msg = i2c_msg.write(0x34, [ord('m'), ord('m'), 127, 180, 0]) # bytes must be in range [0, 256)  x-128
    bus.i2c_rdwr(msg)

In [ ]:
Mouse = {
"LEFT":   1,
"RIGHT":  2,
"MIDDLE": 4,
"ALL":    7
} 

Keyboard = {
# Modifiers
"LEFT_CTRL"		:0x80,
"LEFT_SHIFT"	:0x81,
"LEFT_ALT"		:0x82,
"LEFT_GUI"		:0x83,
"RIGHT_CTRL"	:0x84,
"RIGHT_SHIFT"	:0x85,
"RIGHT_ALT"		:0x86,
"RIGHT_GUI"		:0x87,
# Misckeys
"UP_ARROW"		:0xDA,
"DOWN_ARROW"	:0xD9,
"LEFT_ARROW"	:0xD8,
"RIGHT_ARROW"	:0xD7,
"BACKSPACE"		:0xB2,
"TAB"			:0xB3,
"RETURN"		:0xB0,
"MENU"			:0xED,
"ESC"			:0xB1,
"INSERT"		:0xD1,
"DELETE"		:0xD4,
"PAGE_UP"		:0xD3,
"PAGE_DOWN"		:0xD6,
"HOME"			:0xD2,
"END"			:0xD5,
"CAPS_LOCK"		:0xC1,
"PRINT_SCREEN"	:0xCE,
"SCROLL_LOCK"	:0xCF,
"PAUSE"			:0xD0,
# Numeric keypad
"NUM_LOCK"		:0xDB,
"KP_SLASH"		:0xDC,
"KP_ASTERISK"	:0xDD,
"KP_MINUS"		:0xDE,
"KP_PLUS"		:0xDF,
"KP_ENTER"		:0xE0,
"KP_1"			:0xE1,
"KP_2"			:0xE2,
"KP_3"			:0xE3,
"KP_4"			:0xE4,
"KP_5"			:0xE5,
"KP_6"			:0xE6,
"KP_7"			:0xE7,
"KP_8"			:0xE8,
"KP_9"			:0xE9,
"KP_0"			:0xEA,
"KP_DOT"		:0xEB,
# Functionkeys
"F1"			:0xC2,
"F2"			:0xC3,
"F3"			:0xC4,
"F4"			:0xC5,
"F5"			:0xC6,
"F6"			:0xC7,
"F7"			:0xC8,
"F8"			:0xC9,
"F9"			:0xCA,
"F10"			:0xCB,
"F11"			:0xCC,
"F12"			:0xCD,
"F13"			:0xF0,
"F14"			:0xF1,
"F15"			:0xF2,
"F16"			:0xF3,
"F17"			:0xF4,
"F18"			:0xF5,
"F19"			:0xF6,
"F20"			:0xF7,
"F21"			:0xF8,
"F22"			:0xF9,
"F23"			:0xFA,
"F24"			:0xFB,
}